# How to be a Bioinformagician part 2
- Author: Olga Botvinnik
- Date: 2018-10-09

Abstract: Many computational problems have already been solved and yet hundreds of hours are lost to re-solving them. This series provides tips and tricks that solve common pain points in bioinformatics, using AWS, reading/writing CSVs, extracting data out of file names, and lots more!

## Prerequisities
1. Everything you do should be full screen. Use [divvy](http://mizage.com/divvy/) to make managing windows easier - ask IT for a license
1. [anaconda python](https://www.anaconda.com/download/#macos) - Install the 3.x (e.g. 3.7) version if you don't have it already
1. aws account - Ask Olga or James for one
1. awscli - on the command line, `pip install awscli` after you install Anaconda Python
1. aegea - on the command line, `conda install aegea` after you install Anaconda Python
1. nbconda - on the command line, `conda install nbconda` after you install Anaconda Python
1. reflow. Click this link: https://github.com/grailbio/reflow/releases/download/reflow0.6.8/reflow0.6.8.darwin.amd64 Then on the command line:
    ```
    cd ~/Downloads
    chmod ugo+x reflow0.6.8.darwin.amd64
    sudo mv reflow0.6.8.darwin.amd64 /usr/local/bin/reflow
    ```
    Now the command `reflow` should output a lot of stuff:
    
    ```
      reflow
    The reflow command helps users run Reflow programs, inspect their
    outputs, and query their statuses.

    The command comprises a set of subcommands; the list of supported
    commands can be obtained by running

        reflow -help

    ... (more stuff) ...
    ```
    Then configure reflow, following the [Confluence entry on Reflow (https://czbiohub.atlassian.net/wiki/spaces/DS/pages/838205454/reflow) instructions for configuration:
    ```
    AWS_SDK_LOAD_CONFIG=1 reflow setup-ec2
    AWS_SDK_LOAD_CONFIG=1 reflow setup-s3-repository czbiohub-reflow-quickstart-cache
    AWS_SDK_LOAD_CONFIG=1 reflow setup-dynamodb-assoc czbiohub-reflow-quickstart
    export AWS_REGION=us-west-2
    ```
1. Claim a folder within `s3://czbiohub-cupcakes/` with today's date and your username, e.g.:
    ```
    s3://czbiohub-cupcakes/2018-10-09/olgabot/
    ```
1. [GitHub](http://github.com) username and membership to [@czbiohub](https://github.com/czbiohub/) GitHub group.

Highly recommended:
- If you haven't seen it already, follow https://github.com/czbiohub/codonboarding
- Especially 
    - install homebrew - it makes your life better for installing packages on mac
    ```
    /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
    ```
    - install Oh My ZSH: https://ohmyz.sh/
    - Install exa: https://the.exa.website/ - MUCH easier to install if you installed homebrew.
    ```
    brew install exa
    ```


# How to be a bioinformagician, part 2

# Step 1: Launch Jupyter Notebooks from AWS

## 1.01 Use an existing packer image

### YOU can make your own packer image

I get questions about "how can I make my own image?" and people don't realize it's much easier than they think!

- You aren't beholden to what already exists there
- If the README instructions suck, ask me or James questions and we'll try to answer them. We'll also probably tell you to fix the README for future people with the same question
- Look at the [installation script for this "cupcakes" image](https://github.com/czbiohub/packer-images/blob/master/scripts/cupcakes.sh) - it's super simple. You've already used those commands on AWS so it's not a huge leap to make a packer image.

Here's the `aegea launch` command. You'll want to change the `olgabot` in  `olgabot-cupcakes` to be your own name.


```
aegea launch --ami-tags Name=czbiohub-cupcakes -t t2.xlarge --security-groups='R/RStudio Server and JupyterHub' --iam-role S3fromEC2 --duration-hours 24 olgabot-cupcakes
```

We're using a tiny instance (`t2.xlarge` - see [all instance options](https://www.ec2instances.info/)) because it's cheap and has enough memory (16GiB) for what we need. It'll shut down automatically after 24 hours so make sure to `git add` and `git commit` your changes!!!



#### Aegea launch errors
If you're getting an error that looks like:

```
 Mon  8 Oct - 10:31  ~ 
  aegea launch --ami-tags Name=czbiohub-jupyter -t t2.xlarge --security-groups='R/RStudio Server and JupyterHub' --iam-role S3fromEC2 --duration-hours 24 olgabot-cupcakes
Traceback (most recent call last):
  File "/anaconda3/bin/aegea", line 23, in <module>
    aegea.main()
  File "/anaconda3/lib/python3.6/site-packages/aegea/__init__.py", line 78, in main
    result = parsed_args.entry_point(parsed_args)
  File "/anaconda3/lib/python3.6/site-packages/aegea/launch.py", line 50, in launch
    dns_zone = DNSZone(config.dns.get("private_zone"))
  File "/anaconda3/lib/python3.6/site-packages/aegea/util/aws/__init__.py", line 183, in __init__
    raise AegeaException(msg.format(len(private_zones)))
aegea.util.exceptions.AegeaException: Found 2 private DNS zones; unable to determine zone to use. Set the dns.private_zone key in Aegea config
```


*NOTE: I found this fix by SEARCHING slack for "aegea dns". If you're getting an error, it's likely many other people are, too, so SEARCH slack if you're not getting a response on #eng-support right away*

You can do one of two things, one that will fix the problem forever or a quick fix that will only work once.


##### Edit your aegea config file so it never happens again

```
echo "dns:\n  private_zone: aegea" >> ~/.config/aegea/config.yml
```

##### Fix it just this one time

Add `--no-dns` to your `aegea launch` command before the image name (last argument):

```
aegea launch --ami-tags Name=czbiohub-jupyter -t t2.xlarge --security-groups='R/RStudio Server and JupyterHub' --iam-role S3fromEC2 --duration-hours 24 --no-dns olgabot-cupcakes
```


## 1.02 Log into your instance


```
aegea ssh ubuntu@olgabot-cupcakes
```


## 1.03 Clone my [rcfiles](https://github.com/olgabot/rcfiles) repo and run the setup

I run this one-liner for every single EC2 instance that I make. This way, every instance it has exactly my setup and all my favorite programs, color themes, commands, and aliases. Feel free to fork and edit to your own favorite programs, themes, aliases, etc.

```
git clone https://github.com/olgabot/rcfiles ~/rcfiles && cp ~/rcfiles/Makefile ~ && cd ~ && make
```

The double ampersand (`&&`) means to do the next command only if the previous command was successful, and is a nice way to string together a bunch of stuff in a row, though it *can* be a little unreadable..

If you're curious what the file is happening, the money is in the [Makefile](https://github.com/olgabot/rcfiles/blob/master/Makefile).

### If you get an error `recipe for target 'install' failed`

Sometimes computers are annoying and they lock files to prevent other processes from using them, but they can get locked for too long. Sometimes when you install stuff, you get this error:


```
sudo: unable to resolve host olgabot-cupcakes.aegea
E: Could not get lock /var/lib/dpkg/lock - open (11: Resource temporarily unavailable)
E: Unable to lock the administration directory (/var/lib/dpkg/), is another process using it?
Makefile:4: recipe for target 'install' failed
make: *** [install] Error 100
```

To fix it, do 
```
sudo rm -rf /var/lib/dpkg/lock
```

Then do the last command of the one-liner again, `make`:

```
make
```


## 1.04 If zsh didn't start, type `zsh`

```
zsh
```

## 1.05 Start Screen/Tmux
This will keep Jupyter notebook running forever even if your network connection breaks

Do one of:
```
screen
```
--- OR if you know `tmux` much better ---

```
tmux
```


You may need to type `zsh` again so it starts the Z shell.


## 1.06 Launch jupyter notebook


```
jupyter notebook
```

## 1.07 Open another tab in your terminal with Command-T

Multiple tabs >> (are much better than)  multiple windows because it's much easier to navigate between them

- `Command-Shift-[` moves one tab to the left
- `Command-Shift-]` moves one tab to the right


## 1.08 Tunnel the notebook from AWS to your computer

By default, Jupyter Notebook uses port 8888, and if you have other Jupyter Notebooks running locally, this causes conflict. By rebinding the port to `8899`, then we can have Jupyter Notebooks running both locally and on AWS.

This is the command that binds the remote port `8888` to your local port `8899`.

```
aegea ssh ubuntu@olgabot-cupcakes -NL localhost:8899:localhost:8888 
```

## 1.09 Go to http://localhost:8899 on your laptop

The password is the same as the InnerHub wifi password.


You can also copy the `localhost:8888/?token=asdfasdfadsf` url and replace `8888` with `8899`.

```
 Tue 16 Oct - 01:42  ~ 
  jupyter notebook
[I 01:42:25.272 NotebookApp] Writing notebook server cookie secret to /run/user/1000/jupyter/notebook_cookie_secret
[I 01:42:27.836 NotebookApp] Serving notebooks from local directory: /home/ubuntu
[I 01:42:27.836 NotebookApp] The Jupyter Notebook is running at:
[I 01:42:27.836 NotebookApp] http://localhost:8888/?token=3d81239d82940adfc38110d14c7fc07cb6b3b520ed956e49
[I 01:42:27.836 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[W 01:42:27.836 NotebookApp] No web browser found: could not locate runnable browser.
[C 01:42:27.836 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=3d81239d82940adfc38110d14c7fc07cb6b3b520ed956e49
```

## 1.10 Navigate to the cupcakes/2018 folder

- Open `002_how_to_be_a_bioinformagician_part02.ipynb`

In [ ]:
# Standard convention is to import python standard libraries first, then third-party libraries after that
# See list of standard libraries here: https://docs.python.org/3/library/
# Both import lists should be alphabetically sorted

# --- Python standard library --- #
import glob
import os

# --- Third-party (non-standard Python) libraries --- #
# Interactive visualizations
import holoviews as hv
hv.extension('bokeh')

# Commonly used library for plotting
import matplotlib.pyplot as plt

# Numerical python - arrays, nans
import numpy as np

# python dataframes. very similar to R dataframes
import pandas as pd

# Make the number of characters allowed per column super big since our filenames are long
pd.options.display.max_colwidth = 500

# Reminder to install pyarrow for parquet read/write
import pyarrow

# Reminder to install s3fs to read files from aws
import s3fs

# static visualizations. ggplot2-like
import seaborn as sns

# Nice status bar for 'for' loops
from tqdm import tqdm

# Show figures inside the notebook
%matplotlib inline


# Re-import libraries every time you run a cell. A lifesaver for active development!
%load_ext autoreload
%autoreload 2

In [ ]:
compute_samples = pd.read_csv('lung_cancer/compute/samples.csv')
print(compute_samples.shape)
compute_samples.head()

Cannot have latest version of `awscli` ... there is a bug


    pip install awscli==1.15.83

Check the bucket that we wrote the files to:

In [ ]:
! aws s3 ls s3://olgabot-maca/lung_cancer/sourmash_search/

## Use `aws s3 ls` from within Jupyter

In [ ]:
! aws s3 ls s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/

Now here we'll do the same thing but save the s3 location into the variable `prefix`, and save the output of the `aws s3 ls` to the text file `lung_cancer_sourmash_search.txt`, then use the command line program `cat` to con_*cat*_enate it to standard output so we can see it.

In [ ]:
prefix = 's3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein'
txt = 'lung_cancer_sourmash_search.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

## Read the table we wrote

In [ ]:
aws_s3_ls = pd.read_table(txt, 
                          delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
print(aws_s3_ls.shape)
aws_s3_ls.head()

## Extract out the sample id and whether abundance was on or off in the search

I **always** use [regex101.com](https://regex101.com/r/UGtu79/1) to test out the regular expression and visually inspect that it's doing the right thing.

In [ ]:
pattern = '(?P<sample_id>[\w]+)_ignore-abundance=(?P<ignore_abundance>True|False).csv'

sample_id_abundance = aws_s3_ls.basename.str.extract(pattern)
print(sample_id_abundance.shape)
sample_id_abundance.head()

Look at all sample ids, sorted. There should be two each - one with and without abundance.

In [ ]:
sorted(sample_id_abundance.sample_id.values)

### Combine the sample ids + abundance info and the `aws s3 ls` results

Here, we're adding the `sample_id_abundace` to the right, like "cbind" you may have seen in other languages.

In [ ]:
aws_s3_ls_ids = pd.concat([aws_s3_ls, sample_id_abundance], axis=1)
print(aws_s3_ls_ids.shape)
aws_s3_ls_ids.head()

### Read one of the files *DIRECTLY FROM S3* liek omg omg omg

Here we're demonstrating that we can read csvs (really any text files) **directly** from S3!

In [ ]:
pd.read_csv('s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/D1_B003125_S25_ignore-abundance=False.csv')

Now we're doing the same thing, but using variables because we're good coders.

In [ ]:
csv = f'{prefix}/D1_B003125_S25_ignore-abundance=False.csv'
print("reading:", csv)
df = pd.read_csv(csv)
print(df.shape)
df.head()

Notice that we show the number of rows and columns of the dataframe with `print(df.shape)` and the first 5 rows with `df.head()`. You should do this EVERY single time you transform your dataframe, to make sure what you thought would happen actually happened. I've totally screwed myself over by not double checking at every step so make sure you save yourself!

## Read all the csvs and use `tqdm` to show how fast we're going

[`tqdm`](https://github.com/tqdm/tqdm), meaning "progress" in Arabic, is a super useful tool that shows you how quickly your loop is going. From before, we can see that the size of the `aws_s3_ls` dataframe is 42 and we can watch as the iteration proceeds.  

We'll also use the `%%time` magic to see how long it all takes. You can read more about the built-in Jupyter magics [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html). The key thing to note is that a single percent sign `%` indicates a magic that only operates on that line, e.g. the

    %time df = pd.read_csv(csv)

line, while `%%time` with two percent signs `%` will capture the time of the whole cell. Note that the double percent sign ones must be the FIRST line in the cell.

In [ ]:
%%time

dfs = []

for i, row in tqdm(aws_s3_ls_ids.iterrows()):
    basename = row.basename
    sample_id = basename.split()
    csv = f'{prefix}/{basename}' 
    df = pd.read_csv(csv)
    df['query_sample_id'] = row.sample_id
    df['ignore_abundance'] = row.ignore_abundance
    dfs.append(df)
search_results = pd.concat(dfs, ignore_index=True)
print(search_results.shape)
search_results.head()

In other cases, you get a nice-looking progress bar, but `aws_s3_ls_ids.iterrows()` returns an iterator, not a list so no-one knows the total size *a priori*.

In the example below, you get the nice-looking bar.

In [ ]:
for i in tqdm(range(500)):
    # If i is divisible by 100 with 0 remainder
    if i % 100 == 0:
        print(i)

## Write `extract_metadata.py` file with utility functions that we'll use across notebooks

Now we're going to write a bunch of functions. But because these functions are SO USEFUL we're probably going to need them in different Jupyter Notebooks. So we'll write a file called `extract_metadata.py` using the `%%file` magic, and then we can import it like we import any other package, with `import extract_metadata`.

This is really useful because then you only need to change the code in one place! Rather than having to copy/paste the new function into each file, which is just calling for disaster

In [ ]:
%%file extract_metadata.py
import numpy as np
import pandas as pd

def combine_cell_ontology_free_annotation(row):
    if pd.notnull(row['free_annotation']):
        return '{cell_ontology_class} ({free_annotation})'.format(**row)
    else:
        return row['cell_ontology_class']


def extract_cell_metadata(name_column, pattern='(?P<column>\w+):(?P<value>[\w-]+)'):
    expanded = name_column.str.extractall(pattern)
    expanded_index = expanded.reset_index()
    annotations = expanded_index.pivot(columns='column', values='value', index='level_0')
    
    # Convert "nan" strings to actual NaN objects
    annotations[annotations == "nan"] = np.nan
    annotations['cell_ontology_free_annotation'] = annotations.apply(
        combine_cell_ontology_free_annotation, axis=1)
    return annotations 


def to_key_value_pair(attribute):
    if len(attribute) > 1:
        try:
            return attribute[0], int(attribute[1])
        except ValueError:
            return attribute[0], attribute[1] 
    else:
        return 'comparison_sequence', attribute[0]


def extract_experiment_metadata(basename):
    key = basename.split('.csv')[0]
    split = key.split('_')
    attributes = [x.split('=') for x in split]
    attributes = dict(to_key_value_pair(x) for x in attributes)
    return key, attributes

### Import the file we just wrote and use the `extract_cell_metadata` function

Now we'll import our `extract_metadata.py` file with `import extract_metadata`!

In [ ]:
import extract_metadata

cell_metadata = extract_metadata.extract_cell_metadata(search_results.name)
print(cell_metadata.shape)
cell_metadata.head()

## `join` the search results on the metadata

Notice that the `index` (the `pandas` word for row names) of both are the same - they are the same size and have the same number of rows

In [ ]:
search_results_metadata = search_results.join(cell_metadata)
print(search_results_metadata.shape)
search_results_metadata.head()

## Before we do anything else... SAVE!

We've read in all these files and transformed them a bit. Let's not lose our work!! So we'll write our 

Write the files directly to AWS. This is not as straightforward as just using a filename that starts with `s3://` so I wrote this utility function that we can use across notebooks.

In [ ]:
%%file s3_utils.py
import s3fs

def write_s3(df, filename, fmt='csv', **kwargs):
    fs = s3fs.S3FileSystem(anon=False)
    if fmt == 'csv':
        # csv is a text format
        with fs.open(filename, 'w') as f:
            return df.to_csv(f, **kwargs)
    elif fmt == 'parquet':
        # Parquet is a binary format and needs the "b" flag
        with fs.open(filename, 'wb') as f:
            return df.to_parquet(f, **kwargs)


### What is "parquet"?

[Parquet](https://parquet.apache.org/) is an efficient file format that is very good for **columnar** data storage, think metadata in [tidy data](http://vita.had.co.nz/papers/tidy-data.html) style. 

It's NOT good for storing big, sparse count matrices like gene expression data - use [hdf5](http://docs.h5py.org/en/stable/) for that. Pandas can directly [read/write hdf5](https://pandas.pydata.org/pandas-docs/stable/io.html#hdf5) and [xarray](http://xarray.pydata.org/en/stable/) is a great library for working with tons of labeled, indexed sparse matrices.

Let's take a look at how quickly the search results file gets written in csv vs parquet format, using the `%time` magic to capture the time it takes for a single line.

In [ ]:
import s3_utils

fmt = 'csv'
s3_output_prefix = 's3://olgabot-maca/facs/lung_cancer_v4_metadata'

%time s3_utils.write_s3(search_results_metadata, f'{s3_output_prefix}.{fmt}', fmt=fmt)

In [ ]:
fmt = 'parquet'

%time s3_utils.write_s3(search_results_metadata, f'{s3_output_prefix}.{fmt}', fmt=fmt)

Look at the file size difference!

In [1]:
! aws s3 ls --human-readable s3://olgabot-maca/facs/

                           PRE sourmash/
                           PRE sourmash_compare/
                           PRE sourmash_compare_combined/
                           PRE sourmash_compare_no_track_abundance/
                           PRE sourmash_compare_no_track_abundance_combined/
                           PRE sourmash_compute_all/
                           PRE sourmash_compute_all_b_cells/
                           PRE sourmash_dna-only_trim=false_scaled=100/
                           PRE sourmash_dna-only_trim=false_scaled=1000/
                           PRE sourmash_dna-only_trim=false_scaled=1100/
                           PRE sourmash_dna-only_trim=false_scaled=1200/
                           PRE sourmash_dna-only_trim=false_scaled=1300/
                           PRE sourmash_dna-only_trim=false_scaled=1400/
                           PRE sourmash_dna-only_trim=false_scaled=1500/
                           PRE sourmash_dna-only_trim=false_scaled=1600/
          

## Do some plotting!!! 

Finally, let's make some pictures!

Let's take a look at the overall distribution of the similarity scores using a [`kdeplot`](https://seaborn.pydata.org/generated/seaborn.kdeplot.html) aka a smoothed histogram.

In [ ]:
g = sns.FacetGrid(search_results_metadata)
g.map(sns.kdeplot, 'similarity', shade=True)

### Is similarity different for different target Tabula Muris tissues?

In [ ]:
g = sns.FacetGrid(search_results_metadata, hue='tissue')
g.map(sns.kdeplot, 'similarity', shade=True)
g.add_legend()

### Is similarity different when `ignore_abundance` is True or False?

In [ ]:
g = sns.FacetGrid(search_results_metadata, col='ignore_abundance', hue='tissue')
g.map(sns.kdeplot, 'similarity', shade=True)
g.add_legend()

#### Same plot, but made interactive with [`holoviews`](http://holoviews.org)

In [ ]:
%%opts Distribution [filled=False, tools=['hover'] width=800 height=600] 
%%opts Distribution (line_color=Cycle("Category20"))
%%opts NdOverlay [legend_position='left']


hmap = hv.HoloMap({ignore_abundance: hv.NdOverlay({name: hv.Distribution(df['similarity']) 
              for name, df in abundance_df.groupby('tissue')})
                   for ignore_abundance, abundance_df 
                   in search_results_metadata.groupby('ignore_abundance')}, 
                  kdims=['ignore_abundance'])
hmap.layout('ignore_abundance')

### What about by cell ontology class?

In [ ]:

g = sns.FacetGrid(search_results_metadata, col='ignore_abundance', hue='cell_ontology_class')
g.map(sns.kdeplot, 'similarity', shade=True)
g.add_legend()


## Get the most commonly similar cell ontology class for the best 100 matches

Best = highest similarity, so if we sort by the largest similarity score, we'll have the best!

Use `mode` (remember mean, median and mode?) since we want the celltype that appears the most often in the closest 100 cells.

In [ ]:
grouped = search_results_metadata.groupby(['query_sample_id', 'ignore_abundance'])

cell_ontology_class_guess = grouped.apply( 
    # Need to force `astype(str)` otherwise get an error 
    # of comparing floats (NaNs) to strings
    lambda x: x.nlargest(100, 'similarity')['cell_ontology_class'].mode())
cell_ontology_class_guess

### What about the most common tissues?

In [ ]:
grouped.apply(lambda x: x.nlargest(100, 'similarity')['tissue'].mode())

### Make a dataframe of the cell ontology class guesses

In [ ]:
cell_ontology_class_guess_df = cell_ontology_class_guess.reset_index()
cell_ontology_class_guess_df

### Pivot the dataframe so that "ignore_abundance=True,False" are the columns and the sampel ids are the rows

In [ ]:
cell_ontology_class_guess_pivotted = cell_ontology_class_guess_df.pivot(
    index='query_sample_id', columns='ignore_abundance', values=0)

# Set everything as a string for now
cell_ontology_class_guess_pivotted = cell_ontology_class_guess_pivotted.astype(str)
cell_ontology_class_guess_pivotted

In [ ]:
sorted_cell_ontology_classes = sorted(np.unique(cell_ontology_class_guess_pivotted.astype(str).values.flat))
sorted_cell_ontology_classes

In [ ]:
from sklearn.metrics import confusion_matrix


matrix = confusion_matrix(cell_ontology_class_guess_pivotted['True'], 
                          cell_ontology_class_guess_pivotted['False'],
                         labels=sorted_cell_ontology_classes)
matrix

Make a dataframe because they are better

In [ ]:
confusion_df = pd.DataFrame(matrix, index=sorted_cell_ontology_classes, columns=sorted_cell_ontology_classes)
confusion_df

In [ ]:
sns.heatmap(confusion_df, annot=True, cmap='Purples')

### Now let's make some interactive plots!

In [ ]:
%%opts Distribution [filled=False, tools=['hover'] width=800 height=600] 
%%opts Distribution (line_color=Cycle("Category20"))
%%opts NdOverlay [legend_position='right']


hv.NdOverlay({name: hv.Distribution(df['similarity']) 
              for name, df in search_results_metadata.groupby('tissue')})

Help message from holoviews

In [ ]:
# hv.help(hv.NdOverlay)

In [ ]:
%%opts Distribution [filled=False, tools=['hover'] width=800 height=600] 
%%opts Distribution (line_color=Cycle("Category20"))
%%opts NdOverlay [legend_position='left']


hmap = hv.HoloMap({ignore_abundance: hv.NdOverlay({name: hv.Distribution(df['similarity']) 
              for name, df in abundance_df.groupby('tissue')})
                   for ignore_abundance, abundance_df 
                   in search_results_metadata.groupby('ignore_abundance')}, 
                  kdims=['ignore_abundance'])
hmap.layout('ignore_abundance')

In [ ]:
%%opts Distribution [filled=False, tools=['hover'] width=800 height=600] 
%%opts Distribution (line_color=Cycle())
%%opts NdOverlay [legend_position='right']

groupby = 'cell_ontology_class'

hv.NdOverlay({name: hv.Distribution(df['similarity']) 
              for name, df in search_results_metadata.groupby(groupby)})

## Exercise: make faceted plot where the facets are the "ignore abundance" column and the colors are the cell ontology classes

In [ ]:
# your code goes here

## Look at median similarity per cell ontology per cell

In [ ]:
median_cell_ontology = search_results_metadata.groupby(
    ['query_sample_id', "cell_ontology_class", "ignore_abundance"])['similarity'].median()
median_cell_ontology = median_cell_ontology.unstack(level=0)
# median_cell_ontology = median_cell_ontology.reset_index()

# Set the ignore_abundance level as the outer level for easier subsetting
median_cell_ontology = median_cell_ontology.swaplevel()

# Sort the index
median_cell_ontology = median_cell_ontology.sort_index()

# median_cell_ontology = median_cell_ontology.T
print(median_cell_ontology.shape)
median_cell_ontology.head()

Ignore abundance "True" and "False" are strings so can subset this way:

In [ ]:
median_cell_ontology_yes_abundance = median_cell_ontology.loc["False", :]
median_cell_ontology_yes_abundance.head()

In [ ]:
median_cell_ontology_no_abundance = median_cell_ontology.loc["True", :]
median_cell_ontology_no_abundance.head()

In [ ]:
sns.clustermap(median_cell_ontology_yes_abundance, vmin=0, vmax=1, cmap='Purples')

In [ ]:
sns.clustermap(median_cell_ontology_no_abundance, vmin=0, cmap='Purples')

## Good cell, bad cell

### Good cell

`I3_B003573_S63` seems like a "good cell" that's similar to a lot of things. Let's look at its similarities.

In [ ]:
good_cell = "I3_B003573_S63"

search_results_metadata.query('query_sample_id == @good_cell').groupby('ignore_abundance').apply(lambda x: x.head())

In [ ]:
compute_samples.query('id == @good_cell')

Let's look at its file size

In [ ]:
! aws s3 ls --human-readable s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/$good_cell/

### Bad cell

`D1_B003125_S25` seems like a weird bad cell that's not similar to anything

In [ ]:
bad_cell = "D1_B003125_S25"
search_results_metadata.query('query_sample_id == @bad_cell').head()

In [ ]:
import os

s3_folder = os.path.dirname(compute_samples.query('id == @bad_cell').read1.iloc[0])
s3_folder

In [ ]:
! aws s3 ls --human-readable $s3_folder/

WHOA that's really tiny. Maybe the "good cells" are really ones with deep sequencing?

In [ ]:
median_cell_ontology = median_cell_ontology.sort_values(by=median_cell_ontology.columns.tolist(), ascending=False)
median_cell_ontology.head()

## Seems like the read depth has to do with the similarity here

So let's read all the file sizes for `read1` of the sample id!

In [ ]:
read1s = compute_samples.query('id in @search_results_metadata.query_sample_id').read1.unique()
# print(read1s)
for read1 in read1s:
    ! aws s3 ls --human-readable $read1

This time we'll do the same thing but save the output to a list called `lines`.

In [ ]:
lines = []

for read1 in read1s:
    line = ! aws s3 ls $read1
    lines.append(line[0])

In [ ]:
lines

In [ ]:
read1_txt = '\n'.join(lines)

read1_txt

To pretend that plain strings as "files" for pandas, we use `StringIO`.

In [ ]:
from io import StringIO

read1_aws_s3_ls = pd.read_table(StringIO(read1_txt), delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
read1_aws_s3_ls['sample_id'] = read1_aws_s3_ls['basename'].str.split('_R1').str[0]
read1_aws_s3_ls = read1_aws_s3_ls.set_index('sample_id')
read1_aws_s3_ls

In [ ]:
read1_sizes = read1_aws_s3_ls['bytes']
read1_sizes = read1_sizes.sort_values()
read1_sizes

## Now let's plot the similarity scores, ordered by the read1 sizes

In [ ]:
sns.pointplot(x='query_sample_id', y='similarity', hue='ignore_abundance',
              order=read1_sizes.index, data=search_results_metadata)

Definitely looks like there's a trend

In [ ]:
import seaborn as sns
%matplotlib inline

g = sns.FacetGrid(search_results, hue='query_sample_id', 
                  col='ignore_abundance', sharex=False, sharey=False,
                  hue_order=read1_sizes.index, palette='viridis_r', size=2)
g.map(sns.kdeplot, 'similarity', shade=True)
g.savefig('ashley_lung_cancer_similarity_distribution_per_cell_showing_seq_depth.pdf')

## Commit the changes in a branch so you can see them after your instance is gone

This is also helpful so you can just go to the github website for your branch and look at that for reference!

### To save your changes for the future, create a branch and commit your changes

Since you are saving the output to YOUR own bucket, you'll want to make sure you have the code that made these changes, and the best way to do that is to use `git`.

Create a branch named like this: `yourgithubsername/bioinformagician-part1`, e.g.:

```
git checkout -b olgabot/bioinformagician-part1
```

Add all the files in the `olgas_bioinformagician_tricks` folder:

```
cd ~/code/cupcakes/2018/
git add -A olgas_bioinformagician_tricks
```

Write a message about what files you're committing and why:

```
git commit -m "Use a s3 bucket I can write to"
```

Try to push the changes:

```
git push
```

Then you'll get a "fatal error" (but really nobody died so why the freakout?) that looks like this:


```
fatal: The current branch olgabot/enable_quality_filtering has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin olgabot/enable_quality_filtering
```


Copy/paste THEIR `git push` command which will properly link up your own branch name with the remote branch name on GitHub. (This is what I always do... I'm too lazy to write out the full command myself)

[Check out your branch in the whole tree here!](https://github.com/czbiohub/cupcakes/network)